# | HW3 | MobileNetV2 변형해 보기

**Due: 9/26, 11:59 PM**

- **채점 기준**
  - 아래 과제 설명을 따라야한다.
  - test accuracy가 **80% 이상** 나와야 한다.
- **제출**
  - "HW3_학번_이름.ipynb" 형태로 저장하여 Jupyter Notebook을 그대로 제출.
    - 예: HW3_2022_12345_keondo.ipynb
  - output 지우지 말아 주세요

---

`BatchNormalization(axis, momentum, epsilon)` : https://keras.io/api/layers/normalization_layers/batch_normalization/
- axis: Batch normalization이 적용될 axis. 우리는 채널에 대해서 BN을 적용할 것이다. 
- momentum: Moving average에 적용될 momentum 계수
- epsilon: 0으로 나누는 것을 방지하기 위한 작은 수.


`DepthwiseConv2D(kernel_size, strides, padding, use_bias, depthwise_regularizer)` : https://keras.io/api/layers/convolution_layers/depthwise_convolution2d/

paper:[MobileNetV3](https://openaccess.thecvf.com/content_ICCV_2019/papers/Howard_Searching_for_MobileNetV3_ICCV_2019_paper.pdf)  

이번 과제에서는 MobileNetV3에서 추가된 내용 중 일부를 반영해 볼 것이다. MobilenetV3에서는 모델의 마지막 부분에 아래 그림과 같은 변화가 있었는데, 요약하자면
* Average pooling 앞의 1x1 Convolution layer와 Average pooling layer의 순서를 바꾸어 줌으로써 Computation은 줄이면서 정보의 손실은 최소화하였다.
* 위 변화가 일어나게 됨으로써 그 이전 Inverted residual layer에서 projection/filtering을 해 줄 필요가 없어졌다. 따라서 마지막 Inverted residual layer의 Expansion 이후 바로 Average pooling이 오게 된다.
* 아래 그림을 보면 더 이해가 쉬울 것이다.
<img src="https://user-images.githubusercontent.com/37704174/112775642-734f8a80-9078-11eb-9bc1-a860a1fea407.PNG" width="700" height="700"/> 
* 마지막 Inverted residual layer는 Original last stage 그림에서 맨 앞 세개이다.


<br>
위 내용을 참조하여 Network의 마지막 부분을 변형한 MobileNetV2plus를 구성하라. 위 그림상의 H-swish는 고려하지 않아도 된다.
<img src="https://user-images.githubusercontent.com/37704174/112777027-1229b600-907c-11eb-9f89-a7b61c0843be.PNG" width="700" height="700"/>  

- **채점기준**
  - 위의 변경 사항 반영하기
    - MobileNetV2에서 마지막 inverted residual block 및 뒷부분을 고치면 됨
    - Average pooling의 output의 가로 세로는 1임
  - test accuracy **80%** 이상
    - BatchNormalization, Activation, Dropout, Regularization, Weight initialization 등 자유롭게 수정, 추가, 제거 가능
    - `strides` 수정 가능
    - 나머지는 그대로


## Import Modules

In [1]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
### Q1. Import modules ###

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Add, ReLU, Input, Dense, Activation, Flatten, Conv2D, \
    DepthwiseConv2D, BatchNormalization, GlobalAveragePooling2D, Dropout
from tensorflow.keras.regularizers import l2

/Users/liontea/miniconda3/envs/tf2.9.2_p3.10.5/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
tf.__version__

'2.9.2'

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
"""
Make sure your runtime type is GPU!
"""
physical_devices = tf.config.list_physical_devices('GPU')
print('Num_GPUs:{}, List:{}'.format(len(physical_devices), physical_devices))
gpu_growth = False

if gpu_growth:
    physical_devices = tf.config.list_physical_devices('GPU')
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    except:
        # Invalid device or cannot modify virtual devices once initialized.
        pass

Num_GPUs:1, List:[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Inverted Residual Block

- 실습 때 한 것과 동일: 확인완료

In [4]:
def _inverted_res_block(inputs, expansion, filters, strides):
    momentum = 0.9 # default: 0.99
    regularization = 1e-5 # default : 4e-5
    # momentum = 0.99 # default: 0.99
    # regularization = 2e-5 # default : 4e-5

    x = inputs
    in_chnls = inputs.shape[-1]
    # Expansion
    if expansion != 1:
        x = Conv2D(kernel_size=1, filters=in_chnls * expansion, strides=1, padding='same', use_bias=False, \
            kernel_regularizer=l2(regularization))(x)
        x = BatchNormalization(momentum=momentum, epsilon=0.001)(x)
        x = ReLU(max_value=6)(x)
        x = Dropout(0.8)(x)
        
    # Depthwise convolution
    x = DepthwiseConv2D(kernel_size=3, strides=strides, padding='same', \
        use_bias=False, depthwise_regularizer=l2(regularization))(x)
    x = BatchNormalization(momentum=momentum, epsilon=0.001)(x)
    x = ReLU(max_value=6)(x)
    
    # Linear bottleneck
    x = Conv2D(kernel_size=1, filters=filters, strides=1, padding='same', use_bias=False, \
        kernel_regularizer=l2(regularization))(x)
    x = BatchNormalization(momentum=momentum, epsilon=0.001)(x)
    # No activation
    
    # Residual connection
    if in_chnls == filters and strides == 1:
        x = Add()([inputs, x])
        
    return x #return output of layer

In [5]:
def _inverted_res_block_Last(inputs, expansion, filters, strides):
    # 마지막 inverted resudial layer의 expansion이후 바로 avergae pooling이 오게 됨
    momentum = 0.9 # default: 0.99
    regularization = 1e-5 # default : 4e-5

    x = inputs
    in_chnls = inputs.shape[-1]
    
    # Expansion
    if expansion != 1:
        x = Conv2D(kernel_size=1, filters=in_chnls * expansion, strides=1, padding='same', \
            use_bias=False, kernel_regularizer=l2(regularization))(x)
        x = BatchNormalization(momentum=momentum, epsilon=0.001)(x)
        x = ReLU(max_value=6)(x)

    x = tf.keras.layers.GlobalAveragePooling2D(keepdims=True)(x)

    ### need to be change the dimension to 1 x 1 from Batchsize, 960 -> Batchsize, 1, 1, 960
    x = Conv2D(kernel_size=1, filters=1280, strides=1, padding='same')(x)

    return x #return output of layer

## MobileNetV2 변형 구현

In [6]:
def MobileNetV2plus(input_shape, classes):
    inputs = Input(shape=input_shape)

    momentum = 0.9
    regularizer = 1e-5

    # He Normalizer
    initializers = tf.keras.initializers.HeNormal()

    x = Conv2D(filters=32, kernel_size=3, strides=2, padding='same', \
        kernel_initializer=initializers)(inputs)
    x = BatchNormalization(momentum=momentum, epsilon=0.001)(x)
    x = ReLU(max_value=6)(x)
    
    # inverted residual blocks
    x = _inverted_res_block(inputs = x, expansion=1, filters=16, strides=1)

    x = _inverted_res_block(inputs = x, expansion=6, filters=24, strides=2)
    x = _inverted_res_block(inputs = x, expansion=6, filters=24, strides=1)
    # x = Dropout(0.5)(x)

    x = _inverted_res_block(inputs = x, expansion=6, filters=32, strides=2)
    x = _inverted_res_block(inputs = x, expansion=6, filters=32, strides=1)
    x = _inverted_res_block(inputs = x, expansion=6, filters=32, strides=1)

    x = _inverted_res_block(inputs = x, expansion=6, filters=64, strides=2)
    x = _inverted_res_block(inputs = x, expansion=6, filters=64, strides=1)
    x = _inverted_res_block(inputs = x, expansion=6, filters=64, strides=1)
    x = _inverted_res_block(inputs = x, expansion=6, filters=64, strides=1)
    # x = Dropout(0.8)(x)

    x = _inverted_res_block(inputs = x, expansion=6, filters=96, strides=1)
    x = _inverted_res_block(inputs = x, expansion=6, filters=96, strides=1)
    x = _inverted_res_block(inputs = x, expansion=6, filters=96, strides=1)
    # x = Dropout(0.5)(x)
    
    x = _inverted_res_block(inputs = x, expansion=6, filters=160, strides=2)
    x = _inverted_res_block(inputs = x, expansion=6, filters=160, strides=1)
    x = _inverted_res_block(inputs = x, expansion=6, filters=160, strides=1)
    
    # x = _inverted_res_block_Last(inputs = x, expansion=6, filters=160, strides=1)
    
    # -------------------------------

    in_chnls = x.shape[-1]

    ### Expansion
    expansion = 6
    x = Conv2D(kernel_size=1, filters=in_chnls * expansion, strides=1, padding='same', \
               use_bias=False, kernel_regularizer=l2(regularizer))(x)
    x = BatchNormalization(momentum=momentum, epsilon=0.001)(x)
    x = ReLU(max_value=6)(x)

    ### Average Pooling (None, 1, 1, 960)
    x = tf.keras.layers.GlobalAveragePooling2D(keepdims=True)(x)

    ### 1x1 Conv2D (None, 1, 1, 1280)
    x = Conv2D(kernel_size=1, filters=1280, strides=1, padding='same')(x)

    # -------------------------------
    
    # x = Conv2D(kernel_size=1, filters=1280, strides=1, padding='same')(x)
    # x = BatchNormalization(momentum=momentum, epsilon=0.001)(x)
    # x = ReLU(max_value=6)(x)
    
    # # global average pooling
    # x = GlobalAveragePooling2D()(x)

    # -------------------------------
    
    #3 FC layer
    outputs = Dense(classes, activation='softmax')(x)
    
    return Model(inputs=inputs, outputs=outputs)

In [7]:
my_mobilenet = MobileNetV2plus((32,32,3),classes=10)
# my_mobilenet.summary()

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-09-25 15:19:05.224550: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-25 15:19:05.224691: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Training Data

- keras dataset 혹은 tensorflow dataset 이용
- train data를 9:1로 나눠서 validation data로 이용

In [8]:
from tensorflow.keras.datasets import cifar10
#Load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

#Split train set into train/valid set
from sklearn import model_selection
x_train, x_valid, y_train, y_valid = \
    model_selection.train_test_split(x_train, y_train,test_size=0.1)

## Data Preprocessing
자유롭게 전처리

In [9]:
### Q3. Preporcessing ###

# x_train, x_valid, x_test = x_train / 255.0, x_valid / 255.0, x_test / 255.0

### --------------------------------------------------------------------------------

### Data preprocessing/augmentation for training images
import numpy as np
print ("mean before normalization:", np.mean(x_train)) 
print ("std before normalization:", np.std(x_train))
print("\n")

mean=[0,0,0]
std=[0,0,0]

newX_train = np.ones(x_train.shape)
newX_valid = np.ones(x_valid.shape)
newX_test = np.ones(x_test.shape)

#train, valid, test set에 있는 데이터로 평균과 표준편차를 구함
trainList = [x_train, x_valid, x_test]
for i in range(3):
    mean[i] = np.mean(trainList[i][:,:,:,i])
    std[i] = np.std(trainList[i][:,:,:,i])

#train과 test셋 모두 정규화 작업    
for i in range(3):
    newX_train[:,:,:,i] = x_train[:,:,:,i] - mean[i]
    newX_train[:,:,:,i] = newX_train[:,:,:,i] / std[i]

    newX_valid[:,:,:,i] = x_valid[:,:,:,i] - mean[i]
    newX_valid[:,:,:,i] = newX_valid[:,:,:,i] / std[i]

    newX_test[:,:,:,i] = x_test[:,:,:,i] - mean[i]
    newX_test[:,:,:,i] = newX_test[:,:,:,i] / std[i]
        
x_train = newX_train
x_valid = newX_valid
x_test = newX_test
trainList = [x_train, x_valid, x_test]

for dataSet in trainList:
    print(dataSet.shape)
    print ("mean after normalization:", np.mean(dataSet))
    print ("std after normalization:", np.std(dataSet))
    print(dataSet.max())

# --------------------------------------------------------------------------------

### set up image augmentation
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
    )
    
datagen.fit(x_train)
datagen.fit(x_valid)
datagen.fit(x_test)

# # --------------------------------------------------------------------------------

mean before normalization: 120.76163271122685
std before normalization: 64.22208139002421


(45000, 32, 32, 3)
mean after normalization: -0.00218165795522498
std after normalization: 1.0049587188707239
2.1594935411523557
(5000, 32, 32, 3)
mean after normalization: -0.010582506475374917
std after normalization: 0.9932014938814543
2.1594935411523557
(10000, 32, 32, 3)
mean after normalization: 0.009833931887378109
std after normalization: 1.0024512358578086
2.1594935411523557


## Model Compile
loss function, optimizer 설정

In [10]:
# Learning rate decay function
def decay(epoch):
    ####### 실습 #######
    q = int(epoch / 3)
    return 0.01 * (1 - 0.01)**q
    ###################
    
import math
def exp_decay(epoch):
    ####### 실습 #######
    if epoch < 10:
        return 0.001
    else:
        return 0.001 * math.exp(0.1 * (10-epoch))
    ###################

In [11]:
### Q4. Model compile ###
# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
# my_mobilenet.compile(loss=loss_fn, optimizer='adam', metrics=['accuracy'])

my_mobilenet.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])
#########################

In [12]:
### Q5. Callbacks ###
callbacks = [
    tf.keras.callbacks.LearningRateScheduler(decay), 
    tf.keras.callbacks.EarlyStopping(monitor="loss", patience=7),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=7)
]
#####################

In [13]:
### Callbacks for check point for each 5 epoch ###

# 1. check point path와 checkpoint directory 정의하기

ckpoint_path = "ckpoint_2/cp-{epoch:04d}.ckpt"
ckpoint_dir = os.path.dirname(ckpoint_path)
ckpoint_path = "ckpoint_1/cp.ckpt"

cp_callbacks = tf.keras.callbacks.ModelCheckpoint(
    filepath = ckpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5
)

# cp_callbacks = [
#     tf.keras.callbacks.LearningRateScheduler(exp_decay), 
#     tf.keras.callbacks.EarlyStopping(monitor="loss", patience=5),
#     tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=7),
# ]
#####################

## Model Training
hyperparameter를 적절히 설정한다. (epochs 등..)

In [14]:
### Q6. Training ###
history = my_mobilenet.fit(x_train, y_train, batch_size= 1024,
                          epochs= 30,
                          callbacks=callbacks,              
                          # callbacks=cp_callbacks,
                          validation_data=(x_valid, y_valid),
                          shuffle=True)
####################

Epoch 1/30


2022-09-25 15:19:08.740231: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-25 15:19:10.936950: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


44/44 [==============================] - ETA: 0s - loss: 10.9990 - accuracy: 0.1001

2022-09-25 15:19:28.689798: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


44/44 [==============================] - 21s 361ms/step - loss: 10.9990 - accuracy: 0.1001 - val_loss: 2.4965 - val_accuracy: 0.1069 - lr: 0.0100
Epoch 2/30
44/44 [==============================] - 15s 320ms/step - loss: 2.4095 - accuracy: 0.0995 - val_loss: 2.4066 - val_accuracy: 0.1069 - lr: 0.0100
Epoch 3/30
44/44 [==============================] - 13s 298ms/step - loss: 2.4064 - accuracy: 0.1016 - val_loss: 2.4048 - val_accuracy: 0.0996 - lr: 0.0100
Epoch 4/30
44/44 [==============================] - 14s 327ms/step - loss: 2.4051 - accuracy: 0.1006 - val_loss: 2.4029 - val_accuracy: 0.1069 - lr: 0.0099
Epoch 5/30
44/44 [==============================] - 13s 289ms/step - loss: 2.4032 - accuracy: 0.1008 - val_loss: 2.4016 - val_accuracy: 0.0996 - lr: 0.0099
Epoch 6/30
44/44 [==============================] - 13s 288ms/step - loss: 2.4022 - accuracy: 0.1001 - val_loss: 2.4006 - val_accuracy: 0.0987 - lr: 0.0099
Epoch 7/30
44/44 [==============================] - 13s 293ms/step - loss:

KeyboardInterrupt: 

## 참고용
조교가 학습한 모델의 validation accuracy를 그래프로 나타내 보았다.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['val_accuracy'])
plt.title('Validation Accuracy', fontsize=15)
plt.xlabel('epochs', fontsize=15)
plt.ylabel('Acc.', fontsize=15)

plt.show()

## Test Accuracy

test accuracy 측정 결과 **80% 이상**이 나와야 한다.

In [ ]:
my_mobilenet.evaluate(x_test,y_test)